# Лабораторная работа 3. Кобылкин Константин. Вариант 1

In [49]:
import os
import sys
import json

In [50]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--conf spark.sql.catalogImplementation=in-memory pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --executor-memory 4g --driver-memory 3g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [51]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, FloatType
from pyspark.sql.functions import col, array_contains, to_date, explode, udf, split, when, mean, lit, count
import pyspark.sql.functions as f

# Загрузка данных

## Данные о покупках

In [52]:
schema = StructType() \
      .add("user_id", IntegerType(), True) \
      .add("item_id", IntegerType(), True) \
      .add("purchase", IntegerType(), True)
      
df_user = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/labs/slaba03/laba03_train.csv")

## Справочник телепередач

In [53]:
read_items_schema = StructType(fields=[StructField('item_id', IntegerType()), 
StructField('channel_id', IntegerType()),
StructField('datetime_availability_start', StringType()),
StructField('datetime_availability_stop', StringType()),
StructField('datetime_show_start', StringType()),
StructField('datetime_show_stop', StringType()),
StructField('content_type', IntegerType()),
StructField('title', StringType(), nullable=True),
StructField('year', FloatType(), nullable=True),
StructField('genres', StringType()),
StructField('region_id', IntegerType()),
]) 


df_items = spark.read.format("csv") \
      .option("header", True) \
      .option("sep", "\t")\
      .schema(read_items_schema) \
      .load("/labs/slaba03/laba03_items.csv")

## Данные для теста

In [54]:
schema = StructType() \
      .add("user_id", IntegerType(), True) \
      .add("item_id", IntegerType(), True) 
      
      
df_user_test = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/labs/slaba03/laba03_test.csv")

# Подсчет статистик пользовательского поведения

## Статистики покупаемости

In [55]:
df_user_item_id_with_genres = df_user.join(df_items.select('item_id', 'genres')\
                                                   .filter(df_items.content_type == 1),
                                           on='item_id',
                                           how='left')
genres_words = when(col("genres").isNull(), 'жанр отсутствует').otherwise(col("genres"))
df_user_item_id_with_genres = df_user_item_id_with_genres.withColumn("genres", 
                                                                     genres_words)
df_user_item_id_with_genres = df_user_item_id_with_genres.withColumn('genres_words',
                                                                     split('genres', ','))

In [56]:
df_user_test_item_id_with_genres = df_user_test.join(df_items.select('item_id', 'genres')\
                                                             .filter(df_items.content_type == 1),
                                                     on='item_id',
                                                     how='left')
df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.withColumn("genres", 
                                                                               genres_words)
df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.withColumn('genres_words',
                                                                               split('genres', ','))

In [57]:
df_user_for_statistics = df_user_item_id_with_genres.sampleBy('purchase', 
                                                              fractions={0: 0.3, 1:0.3}, 
                                                              seed=12)

In [58]:
general_purchase = df_user_for_statistics.agg(mean('purchase').alias('general_purchase_rate'))\
                                         .toPandas()\
                                         .at[0, 'general_purchase_rate']

Параметр байесовской регуляризации

In [59]:
n = 5

In [60]:
from pyspark.sql.functions import pandas_udf, PandasUDFType


@pandas_udf(DoubleType(), functionType=PandasUDFType.GROUPED_AGG)
def getBayesianProb(groupby_df):
    m = len(groupby_df)
    k = groupby_df.sum()
    return (k + general_purchase * n) / (m + n) 

In [61]:
user_purchase = df_user_for_statistics.groupby('user_id').agg(getBayesianProb('purchase')\
                                                              .alias('user_purchase_rate')) 
item_id_purchase = df_user_for_statistics.groupby('item_id').agg(getBayesianProb('purchase')\
                                                                 .alias('item_id_purchase_rate'))

In [62]:
genres_words = when(col("genres").isNull(), 
                    'жанр отсутствует').otherwise(col("genres"))
df_user_for_statistics = df_user_for_statistics.withColumn("genres", 
                                                           genres_words)
df_user_for_statistics = df_user_for_statistics.withColumn('genres_words',
                                                           split('genres', ','))

In [63]:
user_genres_purchases = df_user_for_statistics.select('user_id', 'purchase',
                                                      explode("genres_words").alias("single_genre"))
user_genres_purchase_freqs = user_genres_purchases.groupby(['user_id', "single_genre"])\
                                                  .agg(f.sum('purchase').alias('user_sum'),
                                                       count('purchase').alias('user_count'))

In [64]:
user_genres_purchase_freqs = user_genres_purchase_freqs.join(user_purchase,
                                                             on='user_id',
                                                             how='left')
user_genres_purchase_freqs = user_genres_purchase_freqs.withColumn('user_genre_purchase_rate',
                                                                   (col('user_sum') + col('user_purchase_rate') * n) /
                                                                   (col('user_count') + n))
user_genres_purchase_freqs = user_genres_purchase_freqs.drop('user_sum', 'user_count', 
                                                             'user_purchase_rate')

In [65]:
genres_purchase_freqs = user_genres_purchases.groupby(["single_genre"])\
                                             .agg(mean('purchase').alias('genre_purchase_rate'))

## Кодировка жанров

In [66]:
df_comm_items = df_items.select('item_id', 'genres').filter(df_items.content_type == 1)
genres_updated = when(col("genres").isNull(), 
                      'жанр отсутствует').otherwise(col("genres"))
df_comm_items = df_comm_items.withColumn('genres',
                                         genres_updated)
df_comm_items = df_comm_items.withColumn('genres_words',
                                         split('genres', ','))

In [67]:
number_of_genres = df_comm_items.select(explode('genres_words').alias('single_genre')).distinct().count()
number_of_genres

84

In [68]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(binary=True,
                     inputCol="genres_words",
                     outputCol='genres_encoding')
df_comm_items = cv.fit(df_comm_items).transform(df_comm_items)

In [69]:
def split_array_to_list(col):
    def to_list(v):
        return v.toArray().tolist()
    return udf(to_list, ArrayType(DoubleType()))(col)

In [70]:
df_genres_encoding = df_comm_items.select(col('item_id'),
                                          split_array_to_list(col('genres_encoding')).alias("gen_enc"))\
                                  .select([col('item_id')] + [col("gen_enc")[i].alias('gen_enc_' + str(i)) 
                                                              for i in range(84)])

# Добавление вычисленных статистик и признаков в выборки

## Добавление общих статистик  покупаемости и кодировки жанров

In [71]:
features = [(user_purchase, 'user_id'),
            (item_id_purchase, 'item_id'),
            (df_genres_encoding, 'item_id')]

In [72]:
for feature_info in features:
    df_user_item_id_with_genres = df_user_item_id_with_genres.join(feature_info[0],
                                                                   on=feature_info[1],
                                                                   how='left')
    df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.join(feature_info[0],
                                                                             on=feature_info[1],
                                                                             how='left')

## Добавление общих и пользовательских статистик покупаемости жанров

In [73]:
@udf(returnType=StringType())
def getithGenre(genres_list, i):
    if i >= len(genres_list):
        return genres_list[-1]
    else:
        return genres_list[i]

In [74]:
for i in range(3):
    gp = genres_purchase_freqs.withColumnRenamed('single_genre', 
                                                 'single_genre_' + str(i))\
                              .withColumnRenamed('genre_purchase_rate', 
                                                 'genre_purchase_rate_' + str(i))
    ugp = user_genres_purchase_freqs.withColumnRenamed('single_genre', 
                                                       'single_genre_' + str(i))\
                                    .withColumnRenamed('user_genre_purchase_rate', 
                                                       'user_genre_purchase_rate_' + str(i))
    df_user_item_id_with_genres = df_user_item_id_with_genres.withColumn('single_genre_' + str(i),
                                                                         getithGenre('genres_words', 
                                                                                     lit(i)))
    df_user_item_id_with_genres = df_user_item_id_with_genres.join(gp, 
                                                                   on='single_genre_' + str(i),
                                                                   how='left')
    df_user_item_id_with_genres = df_user_item_id_with_genres.join(ugp, 
                                                                   on=['single_genre_' + str(i),
                                                                       'user_id'],
                                                                   how='left')
    df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.withColumn('single_genre_' + str(i),
                                                                                   getithGenre('genres_words', 
                                                                                               lit(i)))
    df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.join(gp, 
                                                                             on='single_genre_' + str(i),
                                                                             how='left')
    df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.join(ugp, 
                                                                             on=['single_genre_' + str(i),
                                                                                 'user_id'],
                                                                             how='left')

In [75]:
df_user_item_id_with_genres = df_user_item_id_with_genres.drop('genres', 'genres_words')
df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.drop('genres', 'genres_words')

## Заполнение пропусков в признаках

In [76]:
user_purchase_rate_updated = when(col("user_purchase_rate").isNull(), 
                                  general_purchase).otherwise(col("user_purchase_rate"))
item_id_purchase_rate_updated = when(col("item_id_purchase_rate").isNull(), 
                                     general_purchase).otherwise(col("item_id_purchase_rate"))

In [77]:
df_user_item_id_with_genres = df_user_item_id_with_genres.withColumn("user_purchase_rate", 
                                                                     user_purchase_rate_updated)
df_user_item_id_with_genres = df_user_item_id_with_genres.withColumn("item_id_purchase_rate", 
                                                                     item_id_purchase_rate_updated)
df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.withColumn("user_purchase_rate", 
                                                                               user_purchase_rate_updated)
df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.withColumn("item_id_purchase_rate", 
                                                                               item_id_purchase_rate_updated)

In [78]:
for i in range(3):
    user_genre_purchase_rate_updated = when(col("user_genre_purchase_rate_" + str(i)).isNull(), 
                                            col('user_purchase_rate')).otherwise(col("user_genre_purchase_rate_" + str(i)))
    df_user_item_id_with_genres = df_user_item_id_with_genres.withColumn("user_genre_purchase_rate_" + str(i), 
                                                                         user_genre_purchase_rate_updated)
    df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.withColumn("user_genre_purchase_rate_" + str(i), 
                                                                                   user_genre_purchase_rate_updated)
    user_genre_purchase_rate_updated_2 = when(col("user_genre_purchase_rate_" + str(i)).isNull(), 
                                            col('genre_purchase_rate_' + str(i))).otherwise(col("user_genre_purchase_rate_" + 
                                                                                                str(i)))
    df_user_item_id_with_genres = df_user_item_id_with_genres.withColumn("user_genre_purchase_rate_" + str(i), 
                                                                         user_genre_purchase_rate_updated_2)
    df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.withColumn("user_genre_purchase_rate_" + str(i), 
                                                                                   user_genre_purchase_rate_updated_2)
    

In [79]:
df_user_item_id_with_genres = df_user_item_id_with_genres.drop('single_genre_0', 'single_genre_1',
                                                               'single_genre_2')
df_user_test_item_id_with_genres = df_user_test_item_id_with_genres.drop('single_genre_0', 'single_genre_1',
                                                                         'single_genre_2')

# Формирование обучающей и валидационной выборки

In [80]:
df_user_train_val = df_user_item_id_with_genres

In [81]:
df_user_train = df_user_train_val.sampleBy('purchase', 
                                           fractions={0: 0.8, 1:0.8}, 
                                           seed=5757)
df_user_val = df_user_train_val.join(df_user_train, 
                                     on=['user_id', 'item_id'], 
                                     how="leftanti")
df_user_test_ = df_user_test_item_id_with_genres

# Обучение модели

In [82]:
from pyspark.ml.classification import GBTClassifier 
from pyspark.ml.feature import VectorAssembler

In [83]:
feats = (['user_purchase_rate', 'item_id_purchase_rate'] + 
         list('gen_enc_' + str(i) for i in range(84)) + 
         list('genre_purchase_rate_' + str(i) for i in range(3)) +
         list('user_genre_purchase_rate_' + str(i) for i in range(3)))

In [84]:
df_user_train = VectorAssembler(inputCols=feats,
                                outputCol='features').transform(df_user_train)
df_user_val = VectorAssembler(inputCols=feats,
                                outputCol='features').transform(df_user_val)
df_user_test_ = VectorAssembler(inputCols=feats,
                                outputCol='features').transform(df_user_test_)

In [85]:
df_user_train = df_user_train.coalesce(3)
df_user_val = df_user_val.coalesce(3)
df_user_test_ = df_user_test_.coalesce(3)

In [86]:
gbtc = GBTClassifier(labelCol="purchase", 
                     maxIter=200,
                     stepSize=0.1,
                     maxDepth=4)
gbtc_model = gbtc.fit(df_user_train)

# Вычисление ROC-AUC на валидации

In [87]:
val_predictions = gbtc_model.transform(df_user_val)

In [88]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", 
                                          labelCol="purchase", 
                                          metricName='areaUnderROC')
evaluator.evaluate(val_predictions)

0.9302829060545991

# Вычисление прогнозов на тестовой выборке

In [89]:
test_predictions = gbtc_model.transform(df_user_test_)

In [92]:
test_predictions_df = test_predictions.select('user_id', 'item_id', 'probability')

In [101]:
first_vector_element=udf(lambda v: float(v[1]),
                         FloatType())

In [104]:
test_predictions_df_ = test_predictions_df.withColumn('purchase',
                                                      firstelement('probability'))\
                                          .orderBy('user_id', 'item_id')\
                                          .drop('probability')

In [105]:
test_predictions_df_.show(5)

+-------+-------+------------+
|user_id|item_id|    purchase|
+-------+-------+------------+
|   1654|    336|0.0061492054|
|   1654|    678|0.0061492054|
|   1654|    691| 0.006198493|
|   1654|    696|0.0064379717|
|   1654|    763|0.0061492054|
+-------+-------+------------+
only showing top 5 rows



# Запись результата в файл

In [107]:
test_predictions_df_.toPandas().to_csv('lab03.csv')

In [ ]:
spark.stop()